In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date
import matplotlib.pyplot as plt
import math
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pylab
from pylab import rcParams

#pd.set_option('display.notebook_repr_html', False)
#pd.set_option('display.max_columns', 8)
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.width', 90)

data_path = '../data/Prod data.xlsx'
fc_path = '../data/fc.xlsx'
#df = pd.read_excel(data_path, index_col=1)  # в качестве индекса используется столбец скважин
df = pd.read_excel(data_path)  # в качестве индекса используется столбец скважин
#df[:5]

Создание словаря параметров скважин

In [2]:
well_dict = {'Plat-2':  {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-5':  {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-6':  {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-8':  {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-13': {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-15': {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-3':  {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-11': {'d': 0.04, 'field': 'Pam-Sas'},
          'Plat-12': {'d': 0.04, 'field': 'Pam-Sas'}
         }
print(well_dict['Plat-2']['d'])
print(well_dict['Plat-5']['field'])
#print(well_dict['Plat-5']['d'],d_dict['Plat-5']['field'])
print('====')
for well, well_info in well_dict.items():
    print(well, ': ', end='')
    
    for param in well_info:
        print(param + ':', well_info[param])
    print('---')

# Create DataFrame from dictionary
df = pd.DataFrame.from_dict(well_dict)
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(fc_path, engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')
writer.save()   # Close the Pandas Excel writer and output the Excel file.
df

0.04
Pam-Sas
====
Plat-2 : d: 0.04
field: Pam-Sas
---
Plat-5 : d: 0.04
field: Pam-Sas
---
Plat-6 : d: 0.04
field: Pam-Sas
---
Plat-8 : d: 0.04
field: Pam-Sas
---
Plat-13 : d: 0.04
field: Pam-Sas
---
Plat-15 : d: 0.04
field: Pam-Sas
---
Plat-3 : d: 0.04
field: Pam-Sas
---
Plat-11 : d: 0.04
field: Pam-Sas
---
Plat-12 : d: 0.04
field: Pam-Sas
---


,Plat-2,Plat-5,Plat-6,Plat-8,Plat-13,Plat-15,Plat-3,Plat-11,Plat-12
d,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04
field,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas


### Извлечение из таблицы последнего исторического дебита

In [4]:
df

,Plat-2,Plat-5,Plat-6,Plat-8,Plat-13,Plat-15,Plat-3,Plat-11,Plat-12
d,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04
field,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas,Pam-Sas


In [3]:
well_name = 'Avil-33'
last_hist_date = '2020-03-01'

# извлечение строчки по скважине на заданную дату
#df[(df['Well'] == well_name) & (df['Date'] == last_hist_date)]

# извлечение месячной добычи по скважине на нужную дату (послдней исторической)
#df['Oil production (actual), t'][(df['Well'] == well_name) & (df['Date'] == last_hist_date)]

# месячная добыча / кол-во часов работы * 24 = среднесуточный дебит
last_hist_rate = df['Oil production (actual), t'][(df['Well'] == well_name) & (df['Date'] == last_hist_date)] \
/ df['Working hours'][(df['Well'] == well_name) & (df['Date'] == last_hist_date)] * 24

# преобразование объекта series в число
last_hist_rate = last_hist_rate.item()

KeyError: 'Oil production (actual), t'

## Создание датафреймов из прогнозных данных

In [ ]:
# X:\MOSCOW\03-WDO\01_All Areas\03 Projects\20 RE\Business plans from WDO\2020.03.25 pdf\Добыча 2020 факт+план.pdf
fc_wdo_20200325 = pd.read_excel(fc_path, sheet_name='WDO_2020.03.25')
fc_wdo_20200325

# X:\MOSCOW\03-WDO\01_All Areas\03 Projects\20 RE\Business plans from WDO\2020.04.10 fc 2020+\Добыча по месторождениям_апрель 2020.xls
fc_wdo_20200410 = pd.read_excel(fc_path, sheet_name='WDO_2020.04.10')
fc_wdo_20200410

fc_wdr_20200403 = pd.read_excel(fc_path, sheet_name='WDR_2020.04.03')
fc_wdr_20200403

### Построение графиков прогнозных данных

In [ ]:
t1 = fc_wdo_20200410['year']  # well_name = 'Plat-2'
data1 = fc_wdo_20200410['wdo']
t2 = fc_wdr_20200403['year']
data2 = fc_wdr_20200403['wdo']

plt.plot(t1, data1, label='FC WDO from 2020.04.10', color="brown")
plt.plot(t2, data2, label='FC WDR from 2020.04.03', color="red")

rcParams['figure.figsize'] = 12, 6

#plt.axis(['2010-01-01', '2024-01-01', 1, 300])
#plt.title(well_name)
#plt.setp( , color="black")
plt.legend()
plt.show()

In [ ]:
fc_wdo_20200410['wdo'].sum().round(1), fc_wdr_20200403['wdo'].sum().round(1)

### Построение таблиц сравнения прогнозов

In [ ]:
merged_fc = pd.concat([fc_wdo_20200410['year'], fc_wdo_20200410['wdo'].round(1), fc_wdr_20200403['wdo'].round(1)],\
                      axis=1, sort=False)
merged_fc.columns = ['year', 'wdo 2020.04.10', 'wdr 2020.04.03']
merged_fc.append(pd.Series(merged_fc.sum(),name='Total')) 
#merged_fc

### Создание кривой падения добычи (по Арпсу)

In [ ]:
qi = last_hist_rate       # начальный дебит = последнему историческому
d = 0.05                  # коэффициент Арпса
q_elim = 3                # экономический предел остановки скважины
s_date_fc = '2020-04-01'  # начальная дата прогноза
# =================================================================================
q_fc = []
month_no = 1  # начинаем считать месяцы с первого (с нулевого если дебит 1-го мес-ца как посл-ий истор-ий)
#q = qi        # 
while qi * math.exp(-month_no * d) > q_elim:
    q = qi * math.exp(-month_no * d)
    q_fc.append(q)
    month_no += 1
# q_fc # массив дебитов (прогнозных)

# генерация массива дат для прогноза
dates_fc = pd.date_range(start = s_date_fc, periods=len(q_fc), freq='MS')

# объединение прогнозных дебитов и массива дат как индекса в объекте Series
q_fc_series = pd.Series(q_fc, index=dates_fc)
#q_fc_series.plot() # построение графика добычи

### Отрисовка исторических данных с прогнозом

In [ ]:
#t = df['Date']  # well_name = 'Plat-2'
data1 = df['Oil production (actual), t'].where(df['Well'] == well_name) / df['Working hours'] * 24
data2 = df['WCT, %'].where(df['Well'] == well_name)
data3 = df['Gas production, M m3'].where(df['Well'] == well_name) / df['Working hours'] * 24

fig, ax1 = plt.subplots(figsize = (10, 5))

color = 'tab:brown'
#ax1.set_xlabel('Date')
ax1.set_ylabel('Daily oil production (t/d)', color=color)
ax1.plot(df['Date'], data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

# кривая прогноза
q_fc_series.plot(color="red")

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Watercut, %', color=color)  # we already handled the x-label with ax1
ax2.plot(df['Date'], data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)


legend = plt.legend(loc='upper left')
plt.setp(legend.get_texts(), color="black")

plt.title(well_name)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
#plt.legend()
plt.show()

### Построение кривой падения добычи в логарифмическом масштабе

In [ ]:
well_name = 'Avil-15'

# Получение посл-го истор-го дебита = месячная добыча / кол-во часов работы * 24 = среднесуточный дебит
last_hist_rate = df['Oil production (actual), t'][(df['Well'] == well_name) & (df['Date'] == last_hist_date)] \
/ df['Working hours'][(df['Well'] == well_name) & (df['Date'] == last_hist_date)] * 24

# преобразование объекта series в число
last_hist_rate = last_hist_rate.item()


qi =  55 #last_hist_rate       # начальный дебит = последнему историческому
d = 0.03                  # коэффициент Арпса
q_elim = 2                # экономический предел остановки скважины
s_date_fc = '2016-03-01'  # начальная дата прогноза
# =================================================================================
q_fc = []
month_no = 1  # начинаем считать месяцы с первого (с нулевого если дебит 1-го мес-ца как посл-ий истор-ий)
#q = qi        # 
while qi * math.exp(-month_no * d) > q_elim:
    q = qi * math.exp(-month_no * d)
    q_fc.append(q)
    month_no += 1
# q_fc # массив дебитов (прогнозных)

dates_fc = pd.date_range(start = s_date_fc, periods=len(q_fc), freq='MS') # генерация массива дат для прогноза
q_fc_series = pd.Series(q_fc, index=dates_fc) # объединение прогнозных дебитов и массива дат как индекса в объекте Series
rcParams['figure.figsize'] = 12, 6
data1 = df['Oil production (actual), t'].where(df['Well'] == well_name) / df['Working hours'] * 24
plt.plot(df['Date'], data1, color="brown")
# кривая прогноза
q_fc_series.plot(color="red")
#q_fc2 = q_fc_series + 2
#q_fc2.plot(color="red")
plt.yscale(value="log")
plt.axis(['2013-04-01', '2025-07-01', 1, 350])
plt.title(well_name)
plt.show()
print(last_hist_rate)
pd.DataFrame(q_fc_series.round(1), dates_fc)

Подбор коэффициетов для прогноза

In [ ]:
well_name = 'Plat-3'
qi = last_hist_rate       # начальный дебит = последнему историческому
d = 0.05                  # коэффициент Арпса
q_elim = 3                # экономический предел остановки скважины
s_date_fc = '2020-04-01'  # начальная дата прогноза
# =================================================================================
q_fc = []
month_no = 1  # начинаем считать месяцы с первого (с нулевого если дебит 1-го мес-ца как посл-ий истор-ий)
#q = qi        # 
while qi * math.exp(-month_no * d) > q_elim:
    q = qi * math.exp(-month_no * d)
    q_fc.append(q)
    month_no += 1
# q_fc # массив дебитов (прогнозных)

# генерация массива дат для прогноза
dates_fc = pd.date_range(start = s_date_fc, periods=len(q_fc), freq='MS')

# объединение прогнозных дебитов и массива дат как индекса в объекте Series
q_fc_series = pd.Series(q_fc, index=dates_fc)


x = df['Date']
y1 = df['Oil production (actual), t'].where(df['Well'] == well_name) / df['Working hours'] * 24
y2 = df['Water production, t'].where(df['Well'] == well_name) / df['Working hours'] * 24
#y3 = ...


colors = ['#C9C19C', '#A9DDFF']
plt.stackplot(x, y1, y2, labels=['Oil production (hist), t/d','Water production (hist), t/d'], colors=colors)
plt.plot(dates_fc, q_fc_series, color="red")
plt.legend(loc='upper left')
plt.title(well_name)
plt.xlim(2010*365, 2024*365)  # нумерация в днях

In [ ]:
#            init_rate  d workover_date workover_rate workover_d
well_dict = {
    'plat-15': {
        'init_rate': 19.4,
                'd': 0.05,
    'workover_date': '2020-08-01',
    'workover_rate': 45,
       'workover_d': 0.05}
}



qi = last_hist_rate       # начальный дебит = последнему историческому
d = 0.05                  # коэффициент Арпса
q_elim = 3                # экономический предел остановки скважины
s_date_fc = '2020-04-01'  # начальная дата прогноза
# =================================================================================
q_fc = []
month_no = 1  # начинаем считать месяцы с первого (с нулевого если дебит 1-го мес-ца как посл-ий истор-ий)
#q = qi        # 
while qi * math.exp(-month_no * d) > q_elim:
    q = qi * math.exp(-month_no * d)
    q_fc.append(q)
    month_no += 1
# q_fc # массив дебитов (прогнозных)

# генерация массива дат для прогноза
dates_fc = pd.date_range(start = s_date_fc, periods=len(q_fc), freq='MS')

# объединение прогнозных дебитов и массива дат как индекса в объекте Series
q_fc_series = pd.Series(q_fc, index=dates_fc)
q_fc_series.plot() # построение графика добычи

# Backup

In [ ]:
plt.figure();
plt.suptitle('Scatter plot')
plt.xlabel('a')
plt.ylabel('b')
plt.scatter(df['Date'], data1)

z = np.polyfit(df['Date'].values.flatten(), data1.values.flatten(), 1)
p = np.poly1d(z)
plt.plot(data1,p(data1),"r--")
plt.title("y=%.6fx+%.6f"%(z[0],z[1])) 

plt.show()

In [ ]:
well_name = 'Plat-2'

plt.figure(figsize = (12, 6))
plt.scatter(df['Date'],
           (df['Oil production (actual), t'].where(df['Well'] == well_name) / df['Working hours'] * 24))
plt.scatter(df['Date'], df['WCT, %'].where(df['Well'] == well_name))
plt.scatter(df['Date'], q_fc)
plt.title('Daily oil production (t/d)')
plt.ylabel('Daily oil production (t/d)')
plt.title(well_name)
plt.show()

In [ ]:
plt.plot(['2010-01-01'] * 100, np.linspace(0, 100, 100), label='median', linestyle='--', linewidth=4)
plt.show()

In [ ]:
#df.dtypes  # типы данных
# Приведение данных к типу данных float
# df['Oil production (actual), t'] = df['Oil production (actual), t'].astype('float')
# df['Working hours'] = df['Working hours'].astype('float')

In [ ]:
df['Date']
well_name = 'Plat-2'
(df['Oil production (actual), t'].where(df['Well'] == well_name)  / df['Working hours'] * 24)[:1] , df['Date']

In [ ]:
df['Date']
well_name = 'Plat-2'
(df['Oil production (actual), t'].where(df['Well'] == well_name)  / df['Working hours'] * 24).tail(5)

In [ ]:
t = df['Date']
well_name = 'Plat-2'
data1 = df['Oil production (actual), t'].where(df['Well'] == well_name) / df['Working hours'] * 24
data2 = df['WCT, %'].where(df['Well'] == well_name)
data3 = df['Gas production, M m3'].where(df['Well'] == well_name) / df['Working hours'] * 24

fig, ax1 = plt.subplots(figsize = (10, 5))

color = 'tab:brown'
#ax1.set_xlabel('Date')
ax1.set_ylabel('Daily oil production (t/d)', color=color)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Watercut, %', color=color)  # we already handled the x-label with ax1
ax2.plot(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

color = 'tab:green'
ax1.plot(t, data3, color=color)

legend = plt.legend(loc='upper left')
plt.setp(legend.get_texts(), color="black")

plt.title(well_name)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
#plt.legend()
plt.show()

In [ ]:
qi = 35       # начальный дебит = последнему историческому
d = 0.06                  # коэффициент Арпса
q_elim = 2                # экономический предел остановки скважины
s_date_fc = '2015-07-01'  # начальная дата прогноза
# =================================================================================
q_fc = []
month_no = 1  # начинаем считать месяцы с первого (с нулевого если дебит 1-го мес-ца как посл-ий истор-ий)
#q = qi        # 
while qi * math.exp(-month_no * d) > q_elim:
    q = qi * math.exp(-month_no * d)
    q_fc.append(q)
    month_no += 1
# q_fc # массив дебитов (прогнозных)

# генерация массива дат для прогноза
dates_fc = pd.date_range(start = s_date_fc, periods=len(q_fc), freq='MS')

# объединение прогнозных дебитов и массива дат как индекса в объекте Series
q_fc_series = pd.Series(q_fc, index=dates_fc)
#q_fc_series.plot() # построение графика добычи

#t = df['Date']  # well_name = 'Plat-2'
data1 = df['Oil production (actual), t'].where(df['Well'] == well_name) / df['Working hours'] * 24
data2 = df['WCT, %'].where(df['Well'] == well_name)
data3 = df['Gas production, M m3'].where(df['Well'] == well_name) / df['Working hours'] * 24
fig, ax1 = plt.subplots(figsize = (10, 5))
color = 'tab:brown'
#ax1.set_xlabel('Date')
ax1.set_ylabel('Daily oil production (t/d)', color=color)
ax1.plot(df['Date'], data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)
# кривая прогноза
q_fc_series.plot(color="red")
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('Watercut, %', color=color)  # we already handled the x-label with ax1
ax2.plot(df['Date'], data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)
plt.title(well_name)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.xlim(2015.5*365, 2024*365)  # нумерация в днях
plt.show()